In [2]:
#tutorial from https://codeclubprojects.org/en-GB/python/iss/
#https://medium.com/@katehayes.m51/tracking-the-international-space-station-a-mini-project-with-geopandas-e682e8a3489f

In [3]:
import time
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import datetime

In [4]:
response = requests.get("http://api.open-notify.org/iss-now.json")

In [5]:
print(response.status_code)

200


In [6]:
#Originally from the API comes as "byte"
response_content = response.content
print(response_content)
print(type(response_content))

b'{"iss_position": {"longitude": "-55.0331", "latitude": "50.5970"}, "message": "success", "timestamp": 1565210227}'
<class 'bytes'>


In [7]:
#Then json.loads converts it to a dictionary
data = json.loads(response_content)
print(data)
print(type(data))

{'iss_position': {'longitude': '-55.0331', 'latitude': '50.5970'}, 'message': 'success', 'timestamp': 1565210227}
<class 'dict'>


In [8]:
#then json_normalize converts it into a pandas object
df = json_normalize(data)
print(df)
print(type(df))

   message   timestamp iss_position.longitude iss_position.latitude
0  success  1565210227               -55.0331               50.5970
<class 'pandas.core.frame.DataFrame'>


In [9]:
df.head()

,message,timestamp,iss_position.longitude,iss_position.latitude
0,success,1565210227,-55.0331,50.5970


In [ ]:
#Create a loop to get ISS info 10 times over every 10 seconds
#response isn't updating and only taking from the first instance
df = pd.DataFrame()
count = 0
while count < 100:
    response = requests.get("http://api.open-notify.org/iss-now.json")
    status = response.status_code
    
    if status != 200:
        print(f"Error improper response code. Code is {status}")
        break
    else:
        response = requests.get("http://api.open-notify.org/iss-now.json")
        resp = response.content
        data_dict = json.loads(resp) # problem is here
        df_temp = json_normalize(data_dict)
        #print(df_temp)
        df = df.append(df_temp)
        time.sleep(10)
        count = count + 1
        print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [ ]:
#drops the unneeded message row
df = df.drop(['message'],axis=1)

In [ ]:
df.columns = df.columns.str.replace("iss_position.","")

In [ ]:
#set the index based on the range of how long the existing index is
df.index = pd.RangeIndex(len(df.index))

In [ ]:
print(df.info())
print()
print(df.shape)

In [ ]:
#need to convert the columns from strings to floats
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)

In [ ]:
timestamp_list = df['timestamp']

In [ ]:
date_list = []

for i in range(len(timestamp_list)):
    temp_date_list = str(datetime.date.fromtimestamp(timestamp_list[i]))
    date_list.append(temp_date_list)

In [ ]:
time_list = []

for i in range(len(timestamp_list)):
    time_list_datetime = datetime.datetime.fromtimestamp(timestamp_list[i])
    time_list_temp = str(datetime.datetime.time(time_list_datetime))
    time_list.append(time_list_temp)

In [ ]:
df['date'] = date_list
df['time'] = time_list
#time list is in miltiary time - hours:minutes:seconds

In [ ]:
df = df.drop('timestamp',axis=1)

In [ ]:
long_list = df['longitude']
lat_list = df['latitude']

In [ ]:
for i in range(len(long_list)):
    print(f"The longitude is {long_list[i]} and the latitude is {lat_list[i]}")

In [ ]:
#import turtle
import turtle

In [ ]:
screen = turtle.Screen()
screen.setup(720,360)
screen.bgpic('map.jpg')
screen.setworldcoordinates(-180,-90,180,90)
#screen.register_shape("iss.png")

In [ ]:
iss = turtle.Turtle()
iss.color("red")
iss.pencolor("red")
iss.pensize("5")

In [ ]:
iss.penup()
iss.goto(long_list[0],lat_list[0])

In [ ]:
iss.pendown()

In [ ]:
for i in range(len(long_list)):
    iss.goto(long_list[i],lat_list[i])

In [ ]:
#closes the turtle program
turtle.exitonclick()